In [1]:
import pandas as pd
import Levenshtein

In [2]:
words = pd.read_csv('words.csv')

In [3]:
words.head()

,Français,Hiragana,Kanji
0,enseignant,きょうし,教師
1,étudiant,がくせい,学生
2,employé d'une compagnie,かいしゃいん,会社員
3,employé (avec nom de compagnie),しゃいん,社員
4,employé de banque,ぎんこういん,銀行員


In [4]:
shuffled_words = words.sample(frac=1).reset_index(drop=True)

In [5]:
def is_similar_japanese_answer(user_answer, correct_answer):
    similarity_threshold = 0.8
    distance = Levenshtein.distance(user_answer.lower(), correct_answer.lower())
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    return similarity >= similarity_threshold

In [6]:
def is_similar_french_answer(user_answer, correct_answer):
    similarity_threshold = 0.7
    if pd.isna(correct_answer) or user_answer == '':
        return False
    user_answer = user_answer.lower()
    correct_answer = correct_answer.lower()
    substring_match = user_answer in correct_answer or correct_answer in user_answer
    if substring_match:
        return True
    distance = Levenshtein.distance(user_answer, correct_answer)
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    
    return similarity >= similarity_threshold

In [7]:
shuffled_sample = shuffled_words.sample(n=40)
shuffled_sample

,Français,Hiragana,Kanji
146,avoir (animal),かいます,飼います
67,réviser sa leçon,ふくしゅうします,復習します
306,autre,ほかの,他の
290,vaisseau spatial,うちゅうせん,宇宙船
313,emprunter,かります,借ります
268,montant/somme d'argent,きんがく,金額
13,salle de cours,きょうしつ,教室
242,annonce / information,おしらせ,お知らせ
289,boîte,はこ,箱
208,librement,じゆうに,自由に


In [8]:
seen_fr = []
correct_count = 0
score_fr = 0
shuffled_sample_jp = shuffled_sample.sample(frac=1).reset_index(drop=True)

for i in range(len(shuffled_sample_jp)):
    if shuffled_sample_jp['Français'].iloc[i] not in seen_fr:
        ask_h = input(f"Comment dire {shuffled_sample_jp['Français'].iloc[i]} en hiragana ?")
        seen_fr.append(shuffled_sample_jp['Français'].iloc[i])

        if is_similar_japanese_answer(ask_h, shuffled_sample_jp['Hiragana'].iloc[i]):
            kanji_value = shuffled_sample_jp['Kanji'].iloc[i]
            if pd.notna(kanji_value):
                ask_k = input("Et en kanji ?")
                if is_similar_japanese_answer(ask_k, kanji_value):
                    correct_count += 1
                    score_fr += 1
                    print('Correct !')
                    print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
                else:
                    print(f"Incorrect ! La bonne réponse est : {kanji_value}")
                    print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
            else:
                correct_count += 1
                score_fr += 1
                print('Correct !')
                print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
        else:
            print(f"Incorrect ! La bonne réponse est : {shuffled_sample_jp['Hiragana'].iloc[i]}")
            print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')

if correct_count == len(shuffled_sample_jp):
    print("Félicitations ! Vous avez répondu correctement à toutes les réponses.")
else:
    print(f"Vous avez répondu correctement à {score_fr} des {len(shuffled_sample_jp)} questions.")

Comment dire quai / voie en hiragana ?
Incorrect ! La bonne réponse est : ばんせん
Bonnes réponses : 0/40
Comment dire [poche] être fixée en hiragana ?かかります
Incorrect ! La bonne réponse est : つきます
Bonnes réponses : 0/40
Comment dire être à l'heure en hiragana ?まにあいます
Et en kanji ?間に合います
Correct !
Bonnes réponses : 1/40
Comment dire [lumière] s'éteindre en hiragana ?けします
Incorrect ! La bonne réponse est : きえます
Bonnes réponses : 1/40
Comment dire parking en hiragana ?
Incorrect ! La bonne réponse est : ちゅうしゃじょう
Bonnes réponses : 1/40
Comment dire vaisseau spatial en hiragana ?
Incorrect ! La bonne réponse est : うちゅうせん
Bonnes réponses : 1/40
Comment dire rendre / rembourser en hiragana ?かします
Et en kanji ?貸します
Incorrect ! La bonne réponse est : 返します
Bonnes réponses : 1/40
Comment dire montant/somme d'argent en hiragana ?
Incorrect ! La bonne réponse est : きんがく
Bonnes réponses : 1/40
Comment dire être en retard en hiragana ?まにあいません
Incorrect ! La bonne réponse est : おくれます
Bonnes réponses : 1/40

In [9]:
seen_k=[]
correct_count = 0
score_jp = 0
shuffled_sample_fr = shuffled_sample.sample(frac=1).reset_index(drop=True)
for i in range(len(shuffled_sample_fr)):
    kanji = shuffled_sample_fr['Kanji'].iloc[i]
    if pd.notna(kanji) and kanji not in seen_k:
        ask_h = input(f"Que veut dire ce kanji : {kanji}?")
        if is_similar_french_answer(ask_h, shuffled_sample_fr['Français'].iloc[i]):
            seen_k.append(kanji)
            correct_count += 1
            score_jp += 1
            print(f"Correct ! C'était bien {shuffled_sample_fr['Français'].iloc[i]}")
            print(f'Bonnes réponses : {score_jp}/{len(shuffled_sample_fr["Kanji"].dropna())}')
        else:
            print(f"Incorrecte ! La bonne réponse était : {shuffled_sample_fr['Français'].iloc[i]}")
            print(f'Bonnes réponses : {score_jp}/{len(shuffled_sample_fr["Kanji"].dropna())}')

if correct_count == len(shuffled_sample_fr):
    print("Félicitations ! Vous avez répondu correctement à toutes les réponses.")
else:
    print(f"Vous avez répondu correctement à {score_jp} des {len(shuffled_sample_fr['Kanji'].dropna())} questions.")

Que veut dire ce kanji : 趣味?hobby
Correct ! C'était bien hobby
Bonnes réponses : 1/38
Que veut dire ce kanji : 社長?président
Correct ! C'était bien président
Bonnes réponses : 2/38
Que veut dire ce kanji : 忙しい?occupé
Correct ! C'était bien occupé / pris
Bonnes réponses : 3/38
Que veut dire ce kanji : 借ります?emprunter
Correct ! C'était bien emprunter
Bonnes réponses : 4/38
Que veut dire ce kanji : 止まります?s'arrêter
Correct ! C'était bien [voiture] s'arrêter
Bonnes réponses : 5/38
Que veut dire ce kanji : 自由に?
Incorrecte ! La bonne réponse était : librement
Bonnes réponses : 5/38
Que veut dire ce kanji : 非常時?
Incorrecte ! La bonne réponse était : cas d'urgence
Bonnes réponses : 5/38
Que veut dire ce kanji : 箱?boite
Correct ! C'était bien boîte
Bonnes réponses : 6/38
Que veut dire ce kanji : 低い?bas
Correct ! C'était bien bas
Bonnes réponses : 7/38
Que veut dire ce kanji : 教室?classe
Incorrecte ! La bonne réponse était : salle de cours
Bonnes réponses : 7/38
Que veut dire ce kanji : 隅?
Incorrect